In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, FixedTicker
import bokeh.palettes as pal
from bokeh.layouts import layout, Spacer, gridplot
output_notebook()

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display

import numpy as np

from bokeh.charts import Bar, Histogram





Loading BokehJS ...

/home/sychen23/.local/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [2]:
from analysis import classify_image

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', 'true', 'true', 'true', 'true', 'true', 'true', 'model_runs/test_new_sigma/classify_log.csv', 'model_runs/test_new_sigma/classifymodel_0.ckpt', 'model_runs/test_new_sigma/classifymodel_', 'model_runs/test_new_sigma/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
analysis.py


In [3]:
b = Button(description="Loading...", icon="arrow", width=400)
dropdown = Dropdown(
    options=['5000', '10000', '50000', '100000', '200000', '300000'],
    value='300000',
    description='Iteration:'
)

figures = list()
iqs = list()
charts = list()

def make_chart(i, j):
#     source = ColumnDataSource(data=dict(data=))
#     bar2 = Bar(data=np.random.rand(10), title="Python Interpreters", plot_width=400, legend=False)
    name = "Machine" if j == 0 else "Human"
    title = "%s Classification %d" % (name, (i + 1))
    p = figure(x_range=(-0.5, 10), y_range=(0, 1), width=200, height=200, tools="")
    
    m = 0.1
    source = ColumnDataSource(data=dict(color=["lime"] * 10, top=np.zeros(10), bottom=np.zeros(10), left=np.arange(10) + m - 0.5, right=np.arange(1, 11) - m - 0.5))
    q = p.quad('left', 'right', 'top', 'bottom', source=source, color="color")


    p.xaxis.axis_label = 'True Class'
    p.yaxis.axis_label = 'Probability'
    
    p.toolbar.logo = None
    p.toolbar_location = None
    p.xaxis[0].ticker=FixedTicker(ticks=np.arange(10))

    
    return p, q

def make_spacer():
    p = figure(x_range=(-0.5, 10), y_range=(0, 1), width=100, height=200, tools="", border_fill_alpha=0, outline_line_alpha=0)
    p.toolbar.logo = None
    p.toolbar_location = None
    p.border_fill_alpha = 0
    p.outline_line_alpha = 0
    
    p.axis.visible = False
    p.grid.visible = False
    
    return p

def make_figure(color, i, j, smol=False):
    if smol:
        w = 12
    else:
        w = 100
        

    name = "Machine" if j == 0 else "Human"
    title = "%s Glimpse %d" % (name, (i + 1))
    p = figure(x_range=(0, w), y_range=(w, 0), width=200, height=200, tools="", title=title)
    
    p.toolbar.logo = None
    p.toolbar_location = None
    p.axis.visible = False

    im = np.zeros((w, w))
    i_source = ColumnDataSource(data=dict(image=[im]))


    iii = p.image(image=[im], x=0, y=w, dw=w, dh=w, palette="Greys256")
    source = ColumnDataSource(data=dict(top=[0], bottom=[0], left=[0], right=[0]))
#     source = ColumnDataSource(data=dict(top=[d["top"]], bottom=[d["bottom"]], left=[d["left"]], right=[d["right"]]))
    q = p.quad('left', 'right', 'top', 'bottom', source=source, color=color, fill_alpha=0, line_width=3)
    
        
    callback = CustomJS(code="""
    console.log(cb_data);
    if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        var i = %d;
        var j = %d;
        if (!this.hovered) {
            cmd = "hover(" + i + ", " + j + ")";
            kernel.execute(cmd, {}, {});
            this.hovered = true;
        }
        
        var that = this;
        
        
        document.querySelectorAll(".bk-plot-layout.bk-layout-fixed").forEach(function(x) {
            x.onmouseleave = function() {
                if (!that.hovered) {
                    return;
                }
                that.hovered = false;
                cmd = "unhover(" + i + ", " + j + ")";
                kernel.execute(cmd, {}, {});
            }
        })
        
    }
    """ % (i, j))
    p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[iii, q]))
    
    
    return p, iii, q;

for i in range(10):
    (machine, i1, q1), (human, i2, q2) = make_figure("pink", i, 0), make_figure("orange", i, 1)
    machine_c, machine_cdata = make_chart(i, 0)
    human_c, human_cdata = make_chart(i, 0)
    figures.append([machine, machine_c, make_spacer(), human, human_c])
    iqs.append([
        (i1, q1),
        (i2, q2)
    ])
    charts.append([machine_cdata, human_cdata])
    
data = None
    
def hover(i, j):
    iqs[i][j][0].data_source.data["image"][0] = data["rs"][i][j]
    iqs[i][j][1].data_source.data = dict(top=[0], bottom=[0], left=[0], right=[0])
    push_notebook(handle=handle)
    
def unhover(i, j):
    iqs[i][j][0].data_source.data["image"][0] = data["img"]
    iqs[i][j][1].data_source.data = data["rects"][i][j]
    push_notebook(handle=handle)
    
    
def update_figures(handle):
    global data
    data = classify_image(int(dropdown.value))
    for i, f in enumerate(figures):
        
        machine, machine_c, spacer, human, human_c = f
        
        (machine_i, machine_q), (human_i, human_q) = iqs[i]
        machine_i.data_source.data["image"][0] = data["img"]#data["rs"][i][0]
        human_i.data_source.data["image"][0] = data["img"] #data["rs"][i][1]
        
        machine_q.data_source.data = data["rects"][i][0]
        human_q.data_source.data = data["rects"][i][1]
        
        machine_cdata, human_cdata = charts[i]
        
        def colorify(ar):
            colors = []
            for x in ar:
                colors.append("lime" if x else "red")
            return colors
        
        clabel = colorify(data["label"])
                
        
        machine_cdata.data_source.data["top"] = data["classifications"][i][0][0]
        machine_cdata.data_source.data["color"] = clabel
        
        human_cdata.data_source.data["top"] = data["classifications"][i][1][0]
        human_cdata.data_source.data["color"] = clabel

                
    push_notebook(handle=handle)
    
def on_click(b):
    b.description = "Loading..."
    update_figures(handle)
    b.description = "Next (Random) Image"
    
b.on_click(on_click)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        on_click(b)

dropdown.observe(on_change)
display(HBox([b, dropdown]))
handle = show(layout(figures), notebook_handle=True)
# update_figures(handle)
on_click(b)

Widget Javascript not detected.  It may not be installed or enabled properly.


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='0d628222-09b2-4715-b5f4-aed9b0f3b919', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='1dc7f0ae-edeb-4f44-8c08-25f854b193be', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='29a82c02-2901-4512-8030-26bb22f320aa', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='38976600-20f9-4508-ae2f-70ec11484370', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='3e58cb7a-712d-4dc2-b051-0ba56ec2e370', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='82a0aede-414d-4818-9e67-70a0fba4d924', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='ac24a6fc-8383-44ca-a2b9-d9034c9c25c8', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='cef8e376-d444-4cf2-890f-eed0c4e43304', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='f8379ac7-13e2-4a14-9ea3-0e0194f14a32', ...)
W

TypeError: classify_image() missing 1 required positional argument: 'new_image'

TypeError: classify_image() missing 1 required positional argument: 'new_image'

TypeError: classify_image() missing 1 required positional argument: 'new_image'

In [6]:
yo = iqs[0][0][0]
yo.data_source.data["image"][0] = np.random.rand(100, 100)
push_notebook(handle=handle)

In [7]:
charts[0][0].data_source.data

{'bottom': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 'color': ['lime',
  'lime',
  'lime',
  'lime',
  'lime',
  'lime',
  'lime',
  'lime',
  'lime',
  'lime'],
 'left': array([-0.4,  0.6,  1.6,  2.6,  3.6,  4.6,  5.6,  6.6,  7.6,  8.6]),
 'right': array([ 0.4,  1.4,  2.4,  3.4,  4.4,  5.4,  6.4,  7.4,  8.4,  9.4]),
 'top': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}

In [8]:
q.data_source.data["top"][0] = 50
push_notebook(handle=handle)

NameError: name 'q' is not defined

In [ ]:
help(layout)

In [ ]:
5 + 5

In [4]:
from pandas import DataFrame

In [5]:
# best support is with data in a format that is table-like
# data = DataFrame({
#     'true class': np.arange(10),
#     'prob': 
# })

# print data


# table-like data results in reconfiguration of the chart with no data manipulation



show(bar2)

NameError: name 'bar2' is not defined

In [ ]:
# best support is with data in a format that is table-like
data = {
    'sample': ['1st', '2nd', '1st', '2nd', '1st', '2nd'],
    'interpreter': ['python', 'python', 'pypy', 'pypy', 'jython', 'jython'],
    'timing': [-2, 5, 12, 40, 22, 30]
}

# x-axis labels pulled from the interpreter column, stacking labels from sample column
bar = Bar(data, values='timing', label='interpreter', stack='sample', agg='mean',
          title="Python Interpreter Sampling", legend='top_right', plot_width=400)

# table-like data results in reconfiguration of the chart with no data manipulation
bar2 = Bar(data, values='timing', label=['interpreter', 'sample'],
           agg='mean', title="Python Interpreters", plot_width=400)

In [ ]:

data = {"a":[4,], "b": [6,], "c":[7,], "d":[2,], "e":[6,]}  ## makes each value a unique category
##data = [4, 6, 7, 2, 6] will make them all the same category

plot2 = Bar(data, title='Unstacked example',
       xlabel='categories', ylabel='values', width=400, height=400,
       tools = "pan,wheel_zoom,box_zoom,reset,resize", palette="Greys256")

show(plot2)

In [ ]:
foobaz = Dropdown(
    options=['5000', '10000', '50000', '100000', '200000', '300000'],
    value='300000',
    description='Number:'
)
display(HBox([b, foobaz]))

In [ ]:

        
foobaz.observe(on_change)